In [7]:
import datetime
from selenium.webdriver.edge.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv

def find_SET_news(n_hours_ago):

    options = Options()
    driver = webdriver.Edge(options=options)
    
    page = 1
    # 新聞網站 URL
    url = "https://www.setn.com/ViewAll.aspx"
    driver.get(url)

    # 初始化一個空字典來存放時間
    news = {}
    index = 1
    
    # 用於存儲已找到的新聞標題
    finded_news_titles = set()

    # 持續滾動頁面,直到找不到符合時間範圍的新聞為止
    while True:
    #for i in range(1):
        # 找到所有新聞項目
        news_items = driver.find_elements(By.CSS_SELECTOR, "div.col-sm-12.newsItems > div")

        # 如果沒有新的新聞項目,退出循環
        if not news_items:
            break

        # 標記本次循環是否找到了符合時間範圍的新聞
        found_new_news = False

        # 遍歷每個新聞項目
        for item in news_items:
            # 獲取標題
            title_element = item.find_element(By.CSS_SELECTOR, "h3.view-li-title > a")
            news_title = title_element.text if title_element else ""

            # 如果標題已經被處理過,跳過
            if news_title in finded_news_titles:
                continue

            # 獲取時間
            time_element = item.find_element(By.CSS_SELECTOR, "time")
            news_time_str = time_element.text if time_element else ""
            news_time = datetime.datetime.strptime(news_time_str, "%m/%d %H:%M")
            news_time = news_time.replace(year=2024)

            # 如果新聞時間在5小時範圍內
            if n_hours_ago <= news_time:
                found_new_news = True
                finded_news_titles.add(news_title)  # 將標題加入已處理集合

                news_link = title_element.get_attribute('href') if title_element else ""
                category_element = item.find_element(By.CSS_SELECTOR, "div.newslabel-tab > a")
                news_category = category_element.text if category_element else ""

                # print(f"標題: {news_title}")
                # print(f"連結: {news_link}")
                # print(f"時間: {news_time_str}")
                # print(f"分類: {news_category}")
                # print("-------------------")

                news[index] = [news_title,
                               news_link,
                               news_time_str,
                               news_category]
                index += 1
            else:
                break
                
        # 如果本次循環中沒有找到符合時間範圍的新聞,就退出
        if not found_new_news:
            break

        # 模擬滾動到頁面底部
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 等待頁面響應

    # 訪問每個新聞詳情頁,爬取內文
    for index in news:
        driver.get(news[index][1])
        content_elements = driver.find_elements(By.CSS_SELECTOR, "p")
        news_content = "\n".join([p.text for p in content_elements])
        news[index].append(news_content)
        # print(f"標題: {news[index][0]}")
        # print(f"內文: {news_content}")

    # 關閉瀏覽器
    driver.quit()
    
    
    # # 將新聞資料存儲為 CSV 檔案
    # with open('SET_news_data.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    #     csv_writer = csv.writer(csvfile)
    #     # 寫入 CSV 標題行
    #     csv_writer.writerow(['標題', '連結', '時間', '分類', '內文'])
    #     # 寫入新聞資料
    #     for index in news:
    #         csv_writer.writerow(news[index])
    return news